<a href="https://colab.research.google.com/github/siva-ee21s125/CS6910---Fundamentals-of-Deep-Learning/blob/main/CS6910_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Load and unzip iNaturalist zip file onto server, then remove zip to optimize performance
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip
print("Extracted the dataset");

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted the dataset


In [2]:
!pip install wandb -qU
import wandb
wandb.login(key='2b25045507d6a89b66edf89be892f3687346ed10')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from glob import glob
import torch
import torchvision.transforms as transforms
import numpy as np
import time

#wandb.init(project='EE21S125_DL_A2')

# Dataset Class for Setting up the data loading process
class PreProcessDataset(Dataset):
    def __init__(self, root_dir, image_size, mode = 'train', augument = 'True'):
        self.root_dir = root_dir;
        self.mode = mode;
        self.augument = augument;
        self.size = image_size;
        self.prepareDataset();
        self.imageTransform();

    def prepareDataset(self):
        self.files = [];
        self.labels = [];
        self.classes = [];

        directory_path = os.path.join(self.root_dir, 'train');
        folders = sorted(os.listdir(directory_path));

        for i in folders:
              if i[0]!= ".":
                self.classes.append(i);
        print("Classes:",self.classes);

        if self.mode == 'train':
            for i in range(len(self.classes)):
                files = glob(os.path.join(directory_path, self.classes[i], '*.jpg'));
                self.labels += [i]*len(files);
                self.files += files;

        elif self.mode == 'test':
            directory_path = os.path.join(self.root_dir, 'val');
            for i in range(len(self.classes)):
                files = glob(os.path.join(directory_path, self.classes[i], '*.jpg'));
                self.labels += [i]*len(files)
                self.files += files
        else:
            print("Invalid Mode");
            return None

    def imageTransform(self):
      if (self.augument):
        self.transform = transforms.Compose([
            transforms.RandomRotation(30),
            transforms.RandomResizedCrop(self.size),
            transforms.RandomHorizontalFlip(),
            #transforms.Resize((256,256)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]);
      else:
        self.transform = transforms.Compose([transforms.Resize(self.size),transforms.ToTensor()]);

        # self.transforms = transforms.Compose([
        #     transforms.RandomRotation(30),
        #     transforms.RandomResizedCrop((256,256)),
        #     transforms.RandomHorizontalFlip(),
        #     transforms.ToTensor(),
        #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #      transforms.Normalize([0.2356, 0.2300, 0.1948], [0.1096, 0.1055, 0.1075])
        # ]);

    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB');
        img = self.transform(img);
        label = self.labels[index];
        label = torch.tensor(label, dtype = torch.long);
        return img, label;

    def __len__(self):
        return len(self.files);

class convolutionNN(torch.nn.Module):
    def __init__(self, image_size, num_input, num_output, filter_organization, batch_norm, num_layers, num_filters, filter_size, maxpool_size, dropout, activation, num_dense_neurons):
        super(convolutionNN,self).__init__();
        self.image_size = image_size;
        self.num_input = num_input;
        self.num_classes = num_output;
        self.filter_organization = filter_organization;
        self.batch_norm = batch_norm;
        self.num_layers = num_layers;
        self.num_filters = num_filters;
        self.filter_size = filter_size;
        self.maxpool_size = maxpool_size;
        self.dropout = dropout;
        self.num_dense_neurons = num_dense_neurons;

        if activation == 'ReLU':
          self.activation = torch.nn.ReLU;
        if activation == 'GELU':
          self.activation = torch.nn.GELU;
        if activation == "SiLU":
          self.activation = torch.nn.SiLU;
        if activation == "Mish":
          self.activation = torch.nn.Mish;
        if activation == 'LeakyReLU':
          self.activation = torch.nn.LeakyReLU;

        self.filter_padding = 1;
        self.filter_stride = 1;
        self.maxpool_stride = 2;
        self.layers = torch.nn.ModuleList();
        filter_sizes_used = [];

        for i in range(self.num_layers):
          #Since filter sizes are reducing significantly
          self.filter_padding = (self.filter_size-1)//2;
          self.convolutionBlock();
          self.num_input = self.num_filters;
          if(self.filter_organization == 'half'):
            self.num_filters = self.num_filters//2;
            filter_sizes_used.append(self.num_input);
          if(self.filter_organization == 'same'):
            self.num_filters = self.num_filters;
            filter_sizes_used.append(self.num_input);
          if(self.filter_organization == 'double'):
            self.num_filters = self.num_filters*2;
            filter_sizes_used.append(self.num_filters);

        final_dim = self.computeInputSizeToDense(filter_sizes_used);
        conv_output_dim = final_dim * final_dim * filter_sizes_used[-1];
        print(final_dim, filter_sizes_used)


        self.FCN = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(conv_output_dim, self.num_dense_neurons),
            self.activation(),
            torch.nn.Dropout(self.dropout),
            torch.nn.Linear(self.num_dense_neurons, self.num_classes)
            );

    def convolutionBlock(self):
        self.layers.append(torch.nn.Conv2d(self.num_input,self.num_filters, self.filter_size, padding = self.filter_padding));
        self.layers.append(self.activation());
        self.layers.append(torch.nn.MaxPool2d(self.maxpool_size, stride = self.maxpool_stride));
        if(self.batch_norm == True):
          self.layers.append(torch.nn.BatchNorm2d(self.num_filters));

    def computeInputSizeToDense(self,filter_sizes_used):
        input = self.image_size;
        print("input:",input)
        for i in range(self.num_layers):
          conv_output = np.ceil((input - self.filter_size + (2*self.filter_padding))/self.filter_stride) + 1;
          #print("Conv output Dim:",conv_output);
          maxpool_output = np.floor(float(conv_output - self.maxpool_size)/self.maxpool_stride) + 1;
          #print("maxpool_output:",maxpool_output);
          input = maxpool_output;
        return int(maxpool_output);

    def forward(self,x):
      for layer in self.layers:
        x = layer(x);
        #print("layer: ", layer, "x.size():", x.size())
      x = self.FCN(x);
      return torch.nn.functional.softmax(x,dim=1);




best_valid_loss = float('inf')
train_loss = []
train_acc = []
val_loss = []
val_acc = []


def accuracy(y_pred, y):
      _, predicted = torch.max(y_pred.data, 1)
      total = y.size(0)
      correct = (predicted == y).sum().item()
      return correct/total

def train(model, dataset, optimizer, criterion, device):
    model.train()
    net_loss=0
    net_acc=0
    for i, (images,labels) in enumerate(dataset):
        #setting gradients to zero again to prevent any incorrect calculation
        optimizer.zero_grad();
        #loading the data accordingly
        images, labels = next(iter(dataset));
        images, labels = images.to(device), labels.to(device);
        #forward pass
        predicted = model.forward(images);
        #calculate loss
        loss = criterion(predicted,labels);
        #gradient descent
        loss.backward();
        #optimizer updates the parameters
        optimizer.step();
        acc = accuracy(predicted,labels);  #calculate accuracy
        net_loss = net_loss + loss.item();
        net_acc = net_acc + acc;
        if((i+1)%10==0):
            print("Step: {}/{}, Loss: {:.4f}, Accuracy: {:.4f}%".format(i+1, len(dataset), net_loss/(i+1), 100*net_acc/(i+1)), end = '\r')
    train_loss.append(net_loss);
    train_acc.append(net_acc);





# def train(self):
#   for config in wandb.config:
#         model = CNN(config.num_filters, config.activation, ...)
#         # Define optimizer, loss function, etc.

#         # Train model
#         # Use WandbCallback for integration with wandb
#         history = model.fit(train_loader, callbacks=[WandbCallback()])

#         # Log metrics to wandb
#         wandb.log({'accuracy': history.history['accuracy'][-1], 'loss': history.history['loss'][-1]})

#     # Run the sweep
#     sweep_id = wandb.sweep(sweep_config, project='EE21S125_DL_A2')
#     wandb.agent(sweep_id, function=train)



In [4]:
def main():

    sweep_config = {
                    'method': 'random',  # Choose 'grid', 'random', 'bayes', etc.
                    'metric': {
                                'name': 'accuracy',
                                'goal': 'maximize'
                              },
                    'parameters': {
                        'num_filters': {'values': [32, 64, 128]},
                        'activation': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish','LeakyReLU']},
                        'filter_organization': {'values': ['same', 'double', 'half']},
                        'data_augmentation': {'values': [True, False]},
                        'batch_normalization': {'values': [True, False]},
                        'num_dense_neurons':{'values': [128,256]},
                        'dropout': {'values': [0.2, 0.3]},
                        'learning_rate': {'values': [1e-4,1e-5,1e-6]},
                        'weight_decay': {'values': [0, 0.0005, 0.5]},
                        'optimizer': {'values': ['NAdam','SGD']},
                        'num_epochs': {'values': [15]}

                    }
                  }

    #Preprocessing Parameters
    augument = wandb.config.data_augmentation;
    num_dense_neurons = wandb.config.num_dense_neurons;
    learning_rate = wandb.config.learning_rate;
    dropout = wandb.config.dropout;
    activation = wandb.config.activation;
    num_filters = wandb.config.num_filters;
    filter_organization = wandb.config.filter_organization;
    batch_norm = wandb.config.batch_normalization;
    num_epochs = wandb.config.num_epochs;

    dataset_directory = 'inaturalist_12K';
    image_size = 128;
    batch_size = 10;
    num_input = 3;
    num_output = 10;
    num_layers = 5;
    filter_size = 3;
    maxpool_size = 2;



    training_set = PreProcessDataset(dataset_directory, image_size, mode, augument);
    train_size = int(0.8*len(training_set));
    validate_size = len(training_set) - train_size;
    train_dataset, validate_dataset = torch.utils.data.random_split(training_set, [train_size, validate_size]);
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0);
    validate_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0);

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');
    print("Device:", device);

    # Creating an instance of the model
    model = convolutionNN(image_size, num_input, num_output, filter_organization, batch_norm, num_layers, num_filters, filter_size, maxpool_size, dropout, activation, num_dense_neurons).to(device);

    criterion = torch.nn.CrossEntropyLoss();    #loss function here is cross entropy (classification problem)
    optimizer=torch.optim.Adam(model.parameters(),learning_rate);



    for epoch in range(epochs):

      start_time = time.time();

      train(model,train_loader,optimizer,criterion,device);
      torch.save(model.state_dict(), 'weights_only.pth');
      #eval(model,valloader,criterion,device)

      #torch.save(model, 'entire_model.pth')

      end_time = time.time();
      elapsed_time = end_time - start_time;

      print("\nTIME TAKEN FOR EPOCH:", elapsed_time);


    print("OVERALL TRAINING COMPLETE");




### Used for checking the transform value ####
    # trainloader = DataLoader(trainset, batch_size=2, shuffle=True, num_workers=0);
    # print("Trainloader:", trainloader);
    # #data_loader = torch.utils.data.DataLoader(trainset.__getitem__(), batch_size=100, shuffle=False)
    # mean = torch.zeros(3)
    # std = torch.zeros(3)
    # num_samples = 0

    # for inputs, _ in trainloader:
    #     batch_size = inputs.size(0)
    #     num_samples += batch_size
    #     # Compute mean across batch and channels
    #     mean += torch.mean(inputs, dim=(0, 2, 3))
    #     # Compute std across batch and channels
    #     std += torch.std(inputs, dim=(0, 2, 3))

    # mean /= num_samples
    # std /= num_samples

    # print("Mean:", mean)
    # print("Standard Deviation:", std)



if __name__ == "__main__":
    main()


OVERALL TRAINING COMPLETE
Classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Device: cuda
input: 256
8 [32, 16, 8, 4, 2]
Step: 800/800, Loss: 2.2793, Accuracy: 14.7875%
TIME TAKEN FOR EPOCH: 232.8966715335846
Step: 800/800, Loss: 2.2631, Accuracy: 17.5875%
TIME TAKEN FOR EPOCH: 211.4577124118805
Step: 800/800, Loss: 2.2407, Accuracy: 20.1250%
TIME TAKEN FOR EPOCH: 209.35363149642944
Step: 800/800, Loss: 2.2444, Accuracy: 19.3625%
TIME TAKEN FOR EPOCH: 208.90216851234436
Step: 800/800, Loss: 2.2252, Accuracy: 22.0125%
TIME TAKEN FOR EPOCH: 209.27525448799133
Step: 800/800, Loss: 2.2258, Accuracy: 21.8125%
TIME TAKEN FOR EPOCH: 209.91004157066345
Step: 800/800, Loss: 2.2182, Accuracy: 22.5125%
TIME TAKEN FOR EPOCH: 208.6268012523651
Step: 800/800, Loss: 2.2190, Accuracy: 22.5750%
TIME TAKEN FOR EPOCH: 208.6104860305786
Step: 800/800, Loss: 2.2165, Accuracy: 22.8375%
TIME TAKEN FOR EPOCH: 221.81998348236084
Step: 800/8

KeyboardInterrupt: 